In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import os.path as osp
from glob import glob
from tqdm.auto import tqdm
import re

import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import torch

import sys
sys.path.append('/root/mediazen/speech2blendshape')

from src.datasets.new_dataset import FaceDataset
from src.datasets.new_datamodule import FaceDataModule
from src.models.deepspeech import Jangnan
from src.utils import plot_spectrogram, plot_result

/root/anaconda3/envs/torch18/lib/python3.8/site-packages/llvmlite/llvmpy/__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
/root/anaconda3/envs/torch18/lib/python3.8/site-packages/llvmlite/llvmpy/core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
/root/anaconda3/envs/torch18/lib/python3.8/site-packages/llvmlite/llvmpy/passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [3]:
data_dir = '/shared/air/shared/youngkim/mediazen/preprocessed/'
essential_dir = osp.join(data_dir, 'essentials')
wav_dir = osp.join(data_dir, 'wav')

essential_paths = [d.path for d in os.scandir(essential_dir)]
wav_paths = [d.path for d in os.scandir(wav_dir)]

len(essential_paths), len(wav_paths)


(2631, 2631)

In [4]:
ignored_columns_16 = ["EyeBlinkLeft", "EyeBlinkRight", "EyeLookDownLeft", "EyeLookDownRight", "EyeLookInLeft", "EyeLookInRight", "EyeLookOutLeft", "EyeLookOutRight", "EyeBlinkLeft", "EyeBlinkRight", "EyeLookDownLeft", "EyeLookDownRight", "EyeLookInLeft", "EyeLookInRight", "EyeLookOutLeft", "EyeLookOutRight", "EyeLookUpLeft", "EyeLookUpRight", "EyeSquintLeft", "EyeSquintRight", "EyeWideLeft", "EyeWideRight", "BrowDownLeft", "BrowDownRight", "BrowInnerUp", "BrowOuterUpLeft", "BrowOuterUpRight", "CheekSquintLeft", "CheekSquintRight", "JawLeft", "JawRight", "MouthLeft", "MouthRight", "MouthUpperUpLeft", "MouthUpperUpRight", "MouthLowerDownLeft", "MouthLowerDownRight", "MouthSmileLeft", "MouthSmileRight", "MouthFrownLeft", "MouthFrownRight", "NoseSneerLeft", "NoseSneerRight", "HeadYaw", "HeadPitch", "HeadRoll", "TongueOut", "LeftEyeYaw", "LeftEyePitch", "LeftEyeRoll", "RightEyeYaw", "RightEyePitch", "RightEyeRoll"]

In [32]:
audio_blob_path = os.path.join(data_dir, 'ggongggong2/audio_ggongggong.pt')
shape_blob_path = os.path.join(data_dir, 'ggongggong2/shape_ggongggong.pt')

sample_rate, indices, audio_data, audio_lengths = torch.load(audio_blob_path)
timecodes, blendshape_count, blendshape_columns, shape_data, shape_lengths, f_names = torch.load(shape_blob_path)
data = list(zip(audio_data, audio_lengths, shape_data, shape_lengths, indices, timecodes, f_names))

In [36]:
file_indices = [int(d.split('_')[0]) for d in f_names]
sentence_nums = [re.sub(r'[^0-9]', '', d.split('_')[2]) for d in f_names]
speaker_names = [re.sub(r'[0-9]+', '', d.split('_')[2]) for d in f_names]

test_sentences = [5, 11, 18, 147, 183]
train_valid_data_names = [d for i, d in enumerate(f_names) if int(sentence_nums[i]) not in test_sentences]
test_data_names = [d for i, d in enumerate(f_names) if int(sentence_nums[i]) in test_sentences]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=1234)
indices = list(range(len(train_valid_data_names)))
train_valid_sentence_nums = [re.sub(r'[^0-9]', '', os.path.basename(d).split('_')[2]) for d in train_valid_data_names]
train_index, valid_index = next(iter(sss.split(indices, train_valid_sentence_nums)))

train_valid_data_names = np.array(train_valid_data_names)

train_data_names = train_valid_data_names[train_index]
valid_data_names = train_valid_data_names[valid_index]

train_file_indices = [int(d.split('_')[0]) for d in train_data_names]
valid_file_indices = [int(d.split('_')[0]) for d in valid_data_names]
test_file_indices = [int(d.split('_')[0]) for d in test_data_names]

train_data = [xx[:4] for xx in data if xx[4].item() in train_file_indices]
valid_data = [xx[:4] for xx in data if xx[4].item() in valid_file_indices]
test_data = [xx for xx in data if xx[4].item() in test_file_indices]

In [37]:
len(train_data), len(train_file_indices), len(valid_data), len(valid_file_indices), len(test_data), len(test_file_indices)

(2308, 2308, 257, 257, 66, 66)

In [39]:
train_data[0]

(tensor([[-0.2306, -0.2395, -0.2595,  ..., -0.2831, -0.2832, -0.2832],
         [-0.2242, -0.1608, -0.2283,  ..., -0.2826, -0.2833, -0.2829],
         [-0.1775, -0.2625, -0.2057,  ..., -0.2821, -0.2827, -0.2830],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor(4055, dtype=torch.int32),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor(2438, dtype=torch.int32))

In [40]:
test_data[0]

(tensor([[-0.2139, -0.2222, -0.2399,  ..., -0.2606, -0.2606, -0.2607],
         [-0.2461, -0.1895, -0.2429,  ..., -0.2602, -0.2604, -0.2606],
         [-0.1222, -0.2132, -0.2113,  ..., -0.2601, -0.2605, -0.2608],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor(6063, dtype=torch.int32),
 tensor([[2.8326e-03, 1.0338e-02, 1.1125e-02,  ..., 0.0000e+00, 0.0000e+00,
          5.0443e-05],
         [2.7040e-03, 1.0201e-02, 1.0967e-02,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [2.5496e-03, 1.0059e-02, 1.0804e-02,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [0.000

In [41]:
import sys
sys.path.append('/root/mediazen/speech2blendshape/src')

from src.datasets.new_dataset import GGongGGongDataset

In [46]:
test_dataset = GGongGGongDataset(test_data)

In [47]:
from torch.utils.data import DataLoader
test_dl = DataLoader(test_dataset, batch_size=32, num_workers=64, pin_memory=False)

In [48]:
for batch in test_dl:
    break

batch

[tensor([[[-0.2139, -0.2222, -0.2399,  ..., -0.2606, -0.2606, -0.2607],
          [-0.2461, -0.1895, -0.2429,  ..., -0.2602, -0.2604, -0.2606],
          [-0.1222, -0.2132, -0.2113,  ..., -0.2601, -0.2605, -0.2608],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.2717, -0.2723, -0.2740,  ..., -0.2784, -0.2785, -0.2785],
          [-0.2301, -0.2535, -0.2662,  ..., -0.2780, -0.2780, -0.2777],
          [-0.2211, -0.2286, -0.2652,  ..., -0.2781, -0.2785, -0.2783],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.1612, -0.3038, -0.2502,  ..., -0.3067, -0.3068, -0.3070],
          [-0.0841, -0.1449,

In [ ]:
# audio_ggongggong
indices = []
spectrograms = []
spectrogram_lengths = []
speakers = []
f_names = []

# shape_ggongggong
timecodes = []
blendshapes = []
blendshape_lengths = []

target_dir = '/shared/youngkim/mediazen/preprocessed/column16/'

for essential in tqdm(os.scandir('/shared/youngkim/mediazen/preprocessed/essentials/')):
    
    f_name = os.path.splitext(essential.name)[0]
    idx = int(f_name.split('_')[0])
    speaker = re.sub(r'[0-9]+', '', f_name.split('_')[2])

    # spectrogram: torch.Tensor (audio_frame, 161)
    # sample_rate: Int, 16000
    # blendshape: Dict (Timecode, BlendShapeCount, *(BlendShapeColumns))
    spectrogram, sample_rate, blendshape = torch.load(essential.path)
    spectrogram_length = len(spectrogram)
    # ignored_columns에 대문자 오타때문에 다시 걸러줌
    for column in ignored_columns_16:
        if column in blendshape.keys():
            del(blendshape[column])
    timecode = blendshape.pop('Timecode') # List (shape_frame)
    blendshape_count = blendshape.pop('BlendShapeCount')[0] # Int, 61 -> 필요없음
    blendshape_columns = list(blendshape.keys()) # List (num. of blendshape)
    try:
        blendshape_tensor = torch.Tensor(list(blendshape.values())).T # torch.Tensor (shape_frame, num. of blendshape)
    except TypeError:
        print('blendshape type error: ', essential.path)
        continue
    blendshape_length = len(blendshape_tensor)
    

    # error check
    if sample_rate != 16000:
        print('sample rate error: ', essential.path)
        continue

    if torch.sum(spectrogram.isnan()):
        print('spectrogram nan error: ', essential.path)
        continue
    
    if torch.sum(blendshape_tensor.isnan()):
        print('blendshape nan error: ', essential.path)
        break

    
    target_file = osp.join(target_dir, f'{f_name}.pt')
    torch.save({
        "idx": idx, # int
        "spectrogram": torch.FloatTensor(spectrogram), # tensor
        "spectrogram_length": spectrogram_length, # int
        "speaker": speaker, # str
        "timecode": torch.FloatTensor(timecode), # tensor
        "blendshape_tensor": torch.FloatTensor(blendshape_tensor), # tensor
        "blendshape_length": blendshape_length, # int
    }, target_file)

0it [00:00, ?it/s]

In [ ]:
blendshape_columns

['JawForward',
 'JawOpen',
 'MouthClose',
 'MouthFunnel',
 'MouthPucker',
 'MouthDimpleLeft',
 'MouthDimpleRight',
 'MouthStretchLeft',
 'MouthStretchRight',
 'MouthRollLower',
 'MouthRollUpper',
 'MouthShrugLower',
 'MouthShrugUpper',
 'MouthPressLeft',
 'MouthPressRight',
 'CheekPuff']

In [ ]:
target_dir = '/shared/youngkim/mediazen/preprocessed/column16/'
# os.mkdir(target_dir)

for i in tqdm(range(len(indices))):
    idx = indices[i]
    spectrogram = torch.FloatTensor(spectrograms[i])
    spectrogram_length = spectrogram_lengths[i]
    speaker = speakers[i]
    timecode = torch.LongTensor([int(time.replace(':', '').replace('.', '')) for time in timecodes[i]])
    blendshape_tensor = torch.FloatTensor(blendshapes[i])
    blendshape_length = blendshape_lengths[i]

    target_file = osp.join(target_dir, f'{f_names[i]}.pt')
    torch.save({
        "idx": idx,
        "spectrogram": spectrogram,
        "spectrogram_length": spectrogram_length,
        "speaker": speaker,
        "timecode": timecode,
        "blendshape_tensor": blendshape_tensor,
        "blendshape_length": blendshape_length,
    }, target_file)


  0%|          | 0/2631 [00:00<?, ?it/s]

In [ ]:
min(spectrogram_lengths), max(spectrogram_lengths)

(3328, 11682)

In [ ]:
sp = set(speakers)
sp, len(sp)

({'AanSeoHue',
  'AnnSeoHue',
  'LeeYooHonew',
  'LeeYuJin',
  'Ohongsuck',
  'SeoHeeJoo',
  'Suljuhwan',
  'bbs',
  'hbinew',
  'hynew',
  'jiha',
  'kmg',
  'kye',
  'lcy',
  'mjki',
  'ohongsuck'},
 16)

In [ ]:
f_names[:3]

['1318_20220831_kye5_0',
 '588_20220809_hynew192_0',
 '256_20220728_LeeYooHonew96_0']

In [ ]:
[d.path for d in os.scandir(target_dir)][:2]

['/shared/youngkim/mediazen/preprocessed/column16/661_20220810_bbs54_0.pt',
 '/shared/youngkim/mediazen/preprocessed/column16/469_20220808_mjki70_0.pt']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data_paths = [d.path for d in os.scandir(target_dir)]
data_paths[0], len(data_paths)

('/shared/youngkim/mediazen/preprocessed/column16/661_20220810_bbs54_0.pt',
 2631)

In [ ]:
file_names = [os.path.basename(d) for d in data_paths]
sentence_nums = [re.sub(r'[^0-9]', '', d.split('_')[2]) for d in file_names]
speaker_names = [re.sub(r'[0-9]+', '', d.split('_')[2]) for d in file_names]
sentence_nums[:10], speaker_names[:10]

(['54', '70', '117', '185', '118', '32', '183', '140', '106', '159'],
 ['bbs',
  'mjki',
  'LeeYuJin',
  'jiha',
  'LeeYuJin',
  'kmg',
  'LeeYooHonew',
  'AnnSeoHue',
  'mjki',
  'LeeYooHonew'])

In [ ]:

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=1234)

data_indices = list(range(len(data_paths)))

test_index, val_index = sss.split(indices, sentence_nums)

264 2367


In [ ]:
test_sentences = [5, 11, 18, 147, 183]

In [ ]:
train_file_names = [d for i, d in enumerate(data_paths) if int(sentence_nums[i]) not in test_sentences]
test_file_names = [d for i, d in enumerate(data_paths) if int(sentence_nums[i]) in test_sentences]

train_file_names[0], len(train_file_names), test_file_names[0], len(test_file_names)

('/shared/youngkim/mediazen/preprocessed/column16/661_20220810_bbs54_0.pt',
 2565,
 '/shared/youngkim/mediazen/preprocessed/column16/284_20220729_LeeYooHonew183_0.pt',
 66)

In [ ]:
len(data_paths)

2631

In [ ]:
data_paths = [d.path for d in os.scandir(target_dir)]
file_names = [os.path.basename(d) for d in data_paths]
sentence_nums = [re.sub(r'[^0-9]', '', d.split('_')[2]) for d in file_names]
speaker_names = [re.sub(r'[0-9]+', '', d.split('_')[2]) for d in file_names]

test_sentences = [5, 11, 18, 147, 183]
train_data_paths = [d for i, d in enumerate(data_paths) if int(sentence_nums[i]) not in test_sentences]
test_data_paths = [d for i, d in enumerate(data_paths) if int(sentence_nums[i]) in test_sentences]


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=1234)
indices = list(range(len(train_data_paths)))
train_sentence_nums = [re.sub(r'[^0-9]', '', os.path.basename(d).split('_')[2]) for d in train_data_paths]
train_index, valid_index = next(iter(sss.split(indices, train_sentence_nums)))

In [ ]:
len(valid_index), len(train_index)

(257, 2308)

In [ ]:
train_index

array([ 950, 1006, 1051, ...,  767, 1610, 2033])

In [ ]:
import numpy as np
np.array(train_data_paths)[train_index]

array(['/shared/youngkim/mediazen/preprocessed/column16/906_20220823_LeeYuJin157_2.pt',
       '/shared/youngkim/mediazen/preprocessed/column16/2221_20220926_lcy64_0.pt',
       '/shared/youngkim/mediazen/preprocessed/column16/2067_20220921_ohongsuck89_0.pt',
       ...,
       '/shared/youngkim/mediazen/preprocessed/column16/621_20220810_bbs15_0.pt',
       '/shared/youngkim/mediazen/preprocessed/column16/33_20220727_hynew37_0.pt',
       '/shared/youngkim/mediazen/preprocessed/column16/2079_20220922_ohongsuck108_0.pt'],
      dtype='<U83')

In [ ]:
data_dir = '/shared/youngkim/mediazen/preprocessed/column16/'

data_paths = [d.path for d in os.scandir(data_dir)]
file_names = [os.path.basename(d) for d in data_paths]
sentence_nums = [re.sub(r'[^0-9]', '', d.split('_')[2]) for d in file_names]
speaker_names = [re.sub(r'[0-9]+', '', d.split('_')[2]) for d in file_names]

test_sentences = [5, 11, 18, 147, 183]
train_data_paths = [d for i, d in enumerate(data_paths) if int(sentence_nums[i]) not in test_sentences]
test_data_paths = [d for i, d in enumerate(data_paths) if int(sentence_nums[i]) in test_sentences]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=1234)
indices = list(range(len(train_data_paths)))
train_sentence_nums = [re.sub(r'[^0-9]', '', os.path.basename(d).split('_')[2]) for d in train_data_paths]
train_index, valid_index = next(iter(sss.split(indices, train_sentence_nums)))

In [ ]:
from dataclasses import dataclass

@dataclass
class Args:
    data_dir: str
    batch_size: int
    num_workers: int
    seed: int

In [ ]:
data_dir = '/shared/youngkim/mediazen/preprocessed/column16/'
args = Args(data_dir, 32, 64, 1234)

In [ ]:
dm = FaceDataModule(args)
dm.prepare_data()
dm.setup(stage='fit')
dm.setup(stage='test')

In [ ]:
sample_train_data = dm.train_dataset[0]
sample_test_data = dm.test_dataset[0]

In [ ]:
sample_train_data.keys(), sample_train_data['f_name']

KeyError: 'f_name'

In [ ]:
sample_test_data.keys(), sample_test_data['f_name']

(dict_keys(['idx', 'spectrogram', 'spectrogram_length', 'speaker', 'timecode', 'blendshape_tensor', 'blendshape_length', 'f_name']),
 '666_20220810_bbs5_0')

In [ ]:
dl_train = dm.train_dataloader()

In [ ]:
for batch in dl_train:
    len(batch)
    break

In [ ]:
len(batch)

4

In [ ]:
batch

[tensor([[[-0.2436, -0.2436, -0.2437,  ..., -0.2448, -0.2448, -0.2448],
          [-0.2381, -0.2119, -0.2163,  ..., -0.2445, -0.2446, -0.2447],
          [ 0.0259, -0.0723, -0.1959,  ..., -0.2441, -0.2446, -0.2444],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.2015, -0.2736, -0.2664,  ..., -0.3019, -0.3011, -0.3011],
          [-0.1477, -0.1160, -0.2527,  ..., -0.3015, -0.3011, -0.3015],
          [ 0.2902,  0.1095, -0.2062,  ..., -0.3014, -0.3016, -0.3013],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.1071, -0.2748, -0.2459,  ..., -0.3299, -0.3300, -0.3298],
          [-0.1263,  0.0014,

In [ ]:
for batch in dm.val_dataloader():
    break

batch

[tensor([[[-0.3279, -0.3343, -0.3444,  ..., -0.3470, -0.3471, -0.3471],
          [-0.2441, -0.2600, -0.3105,  ..., -0.3469, -0.3467, -0.3468],
          [-0.0503, -0.1614, -0.3080,  ..., -0.3467, -0.3464, -0.3468],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.2688, -0.3193, -0.3152,  ..., -0.3227, -0.3226, -0.3226],
          [-0.0879, -0.1808, -0.2829,  ..., -0.3228, -0.3228, -0.3229],
          [-0.2952, -0.0655, -0.2171,  ..., -0.3219, -0.3222, -0.3218],
          ...,
          [-0.3127, -0.0433, -0.1166,  ..., -0.3218, -0.3221, -0.3215],
          [-0.2933, -0.3002, -0.3081,  ..., -0.3225, -0.3227, -0.3228],
          [-0.3176, -0.3191, -0.3226,  ..., -0.3229, -0.3228, -0.3229]],
 
         [[-0.2825, -0.2821, -0.2861,  ..., -0.3055, -0.3055, -0.3055],
          [-0.1059, -0.1585,

In [ ]:
for batch in dm.test_dataloader():
    break

batch

[tensor([[[-0.1261, -0.1603, -0.2297,  ..., -0.2866, -0.2866, -0.2863],
          [ 0.1224,  0.0378, -0.2400,  ..., -0.2858, -0.2860, -0.2855],
          [ 1.1610,  0.7769, -0.0504,  ..., -0.2857, -0.2866, -0.2838],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.2796, -0.2805, -0.2827,  ..., -0.2893, -0.2894, -0.2893],
          [-0.2734, -0.2636, -0.2798,  ..., -0.2891, -0.2893, -0.2893],
          [-0.1937, -0.2255, -0.2797,  ..., -0.2890, -0.2893, -0.2890],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[-0.2450, -0.2500, -0.2595,  ..., -0.2607, -0.2609, -0.2608],
          [-0.0114, -0.1098,

In [ ]:
spec, spec_len, blendshape, blendshape_len, idx, timecode, f_names = batch

In [ ]:
list(map(type, batch))

[torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 torch.Tensor,
 list]

In [ ]:
spec_len

tensor([3460, 4164, 5589, 5086, 6260, 3583, 4902, 9295, 4884, 4649, 3739, 5264,
        6862, 6175, 4208, 6216, 5223, 3616, 6154, 4055, 6534, 4496, 4935, 4055,
        7442, 4043, 4189, 5099, 5448, 4071, 4751, 4414])